In [38]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import Levenshtein as Lv
import time

import numpy as np
import json
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [51]:
fp_uniquebyyear = '../results/dblp-v10-phrases-uniquebyyear.csv'
fp_unique = '../results/dblp-v10-phrases-unique.csv'

In [60]:
df = pd.read_csv(fp_unique, index_col=0)
df

,Phrase Quality,Phrase,Year
0,1.000000,number of,1960
1,1.000000,this paper,1961
2,1.000000,use of,1962
3,1.000000,set of,1964
4,1.000000,analysis of,1964
...,...,...,...
39336,0.603183,great attention,2017
39337,0.602875,limited training,2017
39338,0.602173,public datasets,2017
39339,0.602145,existing works,2017


In [59]:
pd.read_csv(fp_uniquebyyear, index_col=0)

,Phrase Quality,Phrase,Year
0,1.000000,number of,1960
1,1.000000,number of,1961
2,1.000000,this paper,1961
3,1.000000,this paper,1962
4,1.000000,number of,1962
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [61]:
model = LogisticRegression()

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df['Phrase'], df['Year'])

In [45]:
#model.fit(X_train, y_train)

In [46]:
# Can try using correlation between phrases
# Don't have to use a model, just calculate the correlation of input phrases against phrases by year

In [47]:
# Using string similarity
# https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a
